# Setup

In [1]:
!pip install transformers
!pip install datasets
!pip install sentence_transformers
!pip install wordcloud
!git clone https://github.com/pan-webis-de/pan-code.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.6/458.6 kB 42.6 MB/s eta 0:00:00
fatal: destination path 'pan-code' already exists and is not an empty directory.


In [2]:
base_seed = 42

In [3]:
import os
import pandas as pd
import json
from pandas import json_normalize
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_distances
import sys

import torch

import os
sys.path.append(os.path.abspath("pan-code/clef22/style-change-detection/evaluator"))
from evaluator import *
from sentence_transformers import SentenceTransformer, InputExample, util, losses
from torch.utils.data import Dataset, DataLoader
import random
import math




In [4]:
#!unzip pan22.zip


#  Data management

In [4]:
def create_dataframe(directory):
  train_set = []
  for filename in os.listdir(directory):
      f = os.path.join(directory, filename)
      if f.endswith('.json'):
            train_set.append(f)

  train_set.sort(key=lambda f: int(re.sub('\D', '', f))) #sorting alphabetically files by name in order to have the correct order

  data = []
  for name in train_set:
    with open(name) as json_data:
      data_dict = json.load(json_data)
      data.append(data_dict)

  df = json_normalize(data)
  
  train_set_text = []
  for filename in os.listdir(directory):
      f = os.path.join(directory, filename)
      if f.endswith('.txt'):
            train_set_text.append(f)

  train_set_text.sort(key=lambda f: int(re.sub('\D', '', f)))

  text_list = []
  for file in train_set_text:
    with open(file) as f:
      content = f.read()
      text_list.append(content)

  df['input_text'] = text_list

  splitted_text = []
  for text in df['input_text']:
    split = text.splitlines()
    splitted_text.append(split)

  df['splitted_text'] = pd.Series(splitted_text)

  #Sanity check, checking if we have splitted correctly (according to the datset) the input text 
  for i in range(len(df.index)):
    if len(df['splitted_text'].iloc[i]) != len(df['paragraph-authors'].iloc[i]):
      print("we have a problem at row: ",i)
    
  return df

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# Utility functions

In [6]:
def set_reproducibility(seed):
  random.seed(seed)
  np.random.seed(seed)
  
  # Pytorch
  torch.use_deterministic_algorithms(False)
  torch.manual_seed(seed)

In [7]:
def create_contrastive_examples(documents, authors, tokenizer, max_length, max_samples):
    rows = []
    # Splitting data.
    for document, author in zip(documents, authors):
      if len(author) != len(document):
        continue
      for i in range(len(document)):
        for j in range(len(document)):
          if i != j:
            new_row = {}

            text1 = document[i][:sum([len(t) + 1 for t in tokenizer.tokenize(document[i])[:max_length]])]
            text2 = document[j][:sum([len(t) + 1 for t in tokenizer.tokenize(document[j])[:max_length]])]

            new_row["text1"] = text1
            new_row["text2"] = text2

            if author[i] != author[j]:
              new_row["label"] = 0 # 0 if different
            else:
              new_row["label"] = 1 # 1 if same

            rows.append(new_row)

    df = pd.DataFrame(rows, columns=['text1', 'text2', 'label'])

    # Sample the required positive and negative samples

    pos_samples = df[df["label"] == 1]
    neg_samples = df[df["label"] == 0]
    print(len(pos_samples))
    print(len(neg_samples))
    if len(pos_samples) > max_samples//2:
        pos_samples = pos_samples.sample(max_samples//2)
    if len(neg_samples) > max_samples//2:
        neg_samples = neg_samples.sample(max_samples//2)    

    df = pd.concat([pos_samples, neg_samples], axis=0, ignore_index=True).sample(frac=1)

    # From this create a dataset which can be handled by sentence-transformers
    train_examples = []
    for i, row in df.iterrows():
      train_examples.append(InputExample(texts=[row["text1"], row["text2"]], label=row["label"]))
    
    return np.array(train_examples)

In [8]:
# Semi hard/hard negative mining
# Create triplet dataset
def create_triplet_examples(documents, authors, tokenizer, max_length, n_per_document, max_samples, model, margin, mine_all=True, mine_hard=False, mine_semihard = False, is_task1 = False):
  rows = []
  rows_hard = []
  rows_shard = []
  # Splitting data.
  for document, author in zip(documents, authors):
    used_idxs = []
    n_remaining = n_per_document
    
    # Get the indexes of each author
    author_idxs = {}
    for index, value in enumerate(author):
      if value not in author_idxs:
          author_idxs[value] = []
      author_idxs[value].append(index)
    
    if is_task1:
      # Take the hard example (the change one)
      changes = [int(author[i] == author[i+1]) for i in range(len(author)-1)]
      change_idx = changes.index(0)

      try:
        a_idx = random.choice([x for x in author_idxs[author[change_idx]] if x != change_idx])
      except IndexError:
        continue
      pos_idx = change_idx
      neg_idx = change_idx+1

      new_row = {}

      new_row["anchor"] = document[a_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(document[a_idx])[:max_length]])]
      new_row["pos"] = document[pos_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(document[pos_idx])[:max_length]])]
      new_row["neg"] = document[neg_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(document[neg_idx])[:max_length]])]

      rows_sh.append(new_row)
      used_idxs.append((a_idx, pos_idx, neg_idx))

      n_remaining -= 1
    else:
      if all([len(author_idxs[k])==1 for k in author_idxs]):
          continue

    # I could compute how many combinations are possible but it's too hard
    idx_errors = 0
    while n_remaining > 0:
      if idx_errors > 5:
        break
      try:
        # Sample a random author
        anchor_author = random.choice(list(author_idxs.keys()))

        # Get a paragraph from that author (anchor)
        a_idx = random.choice(author_idxs[anchor_author])

        # Get another paragraph from the same author
        pos_idx = random.choice([x for x in author_idxs[anchor_author] if x != a_idx])

        # Sample another author
        neg_author = random.choice([x for x in list(author_idxs.keys()) if x != anchor_author])

        # Get a paragraph from that author
        neg_idx = random.choice(author_idxs[neg_author])
      except IndexError:
        idx_errors += 1
        continue

      if (a_idx, pos_idx, neg_idx) in used_idxs:
        idx_errors += 1
        continue

      else:
        used_idxs.append((a_idx, pos_idx, neg_idx))
        new_row = {}

        new_row["anchor"] = document[a_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(document[a_idx])[:max_length]])]
        new_row["pos"] = document[pos_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(document[pos_idx])[:max_length]])]
        new_row["neg"] = document[neg_idx][:sum([len(t) + 1 for t in tokenizer.tokenize(document[neg_idx])[:max_length]])]

        if mine_hard or mine_semihard:
          # Compute embeddings
          a_embedding = model.encode([new_row["anchor"]], convert_to_tensor=True)
          p_embedding = model.encode([new_row["pos"]], convert_to_tensor=True)
          n_embedding = model.encode([new_row["neg"]], convert_to_tensor=True)

          # Compute distance between anchor and positive
          a_p_dist = 1 - util.cos_sim(a_embedding, p_embedding).cpu().item()
          # Compute distance between anchor and negative
          a_n_dist = 1 - util.cos_sim(a_embedding, n_embedding).cpu().item()

          if a_n_dist < a_p_dist:
            # Hard example
            if mine_hard:
              rows_hard.append(new_row)
          elif a_n_dist < a_p_dist + margin and a_n_dist > a_p_dist:
            # Semi hard example
            if mine_semihard:
              rows_shard.append(new_row)
          else:
            if mine_all:
              rows.append(new_row)
          n_remaining -= 1
        else:
          rows.append(new_row)
          n_remaining -= 1
  print(f"N easy:{len(rows)}, N hard: {len(rows_hard)}, N semihard:{len(rows_shard)}")
  random.shuffle(rows_shard)
  random.shuffle(rows_hard)
  random.shuffle(rows)

  all_rows = [*rows_shard, *rows_hard, *rows]
  if len(all_rows) < max_samples:
    all_rows = all_rows[:max_samples]
  print(len(all_rows))

  df = pd.DataFrame(all_rows, columns=['anchor', 'pos', 'neg'])

  # From this create a dataset which can be handled by sentence-transformers
  train_examples = []
  for i, row in df.iterrows():
    train_examples.append(InputExample(texts=[row["anchor"], row["pos"], row["neg"]]))
  
  return np.array(train_examples)

In [9]:
# Compute Same class / Diff class similarities

def compute_same_diff_contrastive(model, dataloader):
  model = model.to(device)
  model.eval()
  same_sims = []
  diff_sims = []

  for idx, batch in enumerate(dataloader):
      author_info = batch[1]

      with torch.no_grad():
          text1 = {k:batch[0][0][k].to(device) for k in batch[0][0]}
          embeddings1 = model(text1)["sentence_embedding"]

          text2 = {k:batch[0][1][k].to(device) for k in batch[0][1]}
          embeddings2 = model(text2)["sentence_embedding"]
          
      
      for i in range(len(author_info)):
          same = author_info[i]
          sim = util.cos_sim(embeddings1[i], embeddings2[i]).cpu().item()
          if same == 1:
              same_sims.append(sim)
          else:
              diff_sims.append(sim)
  return same_sims, diff_sims

def compute_same_diff_triplet(model, dataloader):
  model.eval()
  same_sims = []
  diff_sims = []

  for idx, batch in enumerate(dataloader):
      with torch.no_grad():
          anchor = {k:batch[0][0][k].to(device) for k in batch[0][0]}
          embeddings_a = model(anchor)["sentence_embedding"]

          pos = {k:batch[0][1][k].to(device) for k in batch[0][1]}
          embeddings_p = model(pos)["sentence_embedding"]

          neg = {k:batch[0][2][k].to(device) for k in batch[0][2]}
          embeddings_n = model(neg)["sentence_embedding"]

      for i in range(anchor["input_ids"].shape[0]):

          sim_pos = util.cos_sim(embeddings_a[i], embeddings_p[i]).cpu().item()
          sim_neg = util.cos_sim(embeddings_a[i], embeddings_n[i]).cpu().item()

          same_sims.append(sim_pos)
          
          diff_sims.append(sim_neg)
  return same_sims, diff_sims

# Task 1

In [18]:
train_directory = './dataset1/train'
df_train = create_dataframe(train_directory)
df_train.head()

valid_directory = './dataset1/validation'
df_valid = create_dataframe(valid_directory)
df_valid.head()

,authors,site,multi-author,changes,paragraph-authors,input_text,splitted_text
0,2,serverfault.com.7z,1,"[0, 0, 1, 0, 0]","[1, 1, 1, 2, 2, 2]",If you can handle a slight delay in the data y...,[If you can handle a slight delay in the data ...
1,2,serverfault.com.7z,1,"[0, 1, 0, 0, 0, 0]","[1, 1, 2, 2, 2, 2, 2]",I figured i'd share my method which worked to ...,[I figured i'd share my method which worked to...
2,2,networkengineering.stackexchange.com.7z,1,"[1, 0, 0, 0]","[1, 2, 2, 2, 2]",This is not possible the way I want to impleme...,[This is not possible the way I want to implem...
3,2,serverfault.com.7z,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 2, 2, 2, 2, 2]",We have central HQ building and a lot of small...,[We have central HQ building and a lot of smal...
4,2,cstheory.stackexchange.com.7z,1,"[1, 0, 0]","[1, 2, 2, 2]",Moser's proof of the constructive Lovasz Local...,[Moser's proof of the constructive Lovasz Loca...


In [24]:
def prediction_task1(model, sentences):
  # Get metric embeddings
  embeddings = model.encode(sentences, convert_to_tensor=True)

  similarity_scores = []
  # For each successive embedding find their similarity
  for i in range(len(sentences)-1):
    sim = util.cos_sim(embeddings[i], embeddings[i+1])
    similarity_scores.append(sim.item()) 
  
  # Puy the change at the position where similarity is lowest
  change_idx = np.argmin(similarity_scores)

  out_changes = np.zeros(len(similarity_scores))
  out_changes[change_idx] = 1
  return out_changes

# Evaluator function
def evaluate_mmodel_task1(model : SentenceTransformer, df):

  valid_labels = []
  y_preds = []

  for i, row in df.iterrows():
    sentences = row["splitted_text"]

    out_changes = prediction_task1(model, sentences)
    valid_labels += row["changes"]

    y_preds += list(out_changes)


  print(classification_report(valid_labels, y_preds))


## Model 1 - Frozen Sbert + cosine sim

In [13]:
set_reproducibility(base_seed)
# Instantiate model
#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)


In [14]:
evaluate_mmodel_task1(model, df_valid)

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      1841
           1       0.48      0.48      0.48       300

    accuracy                           0.85      2141
   macro avg       0.70      0.70      0.70      2141
weighted avg       0.85      0.85      0.85      2141



## Model 2 - Trained SBERT with contrastive loss 

In [15]:
set_reproducibility(base_seed)
# Instantiate model

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [16]:
max_length = 128
train_ds = create_contrastive_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, max_length, max_samples = 20000)
print("Dataset size:", len(train_ds))

train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

# Define contrastive loss
contrastive_loss = losses.ContrastiveLoss(model=model, distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE, margin=0.6)
epochs = 2

# Train
model.fit(train_objectives=[(train_dl, contrastive_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       }
        )


62682
39578
Dataset size: 20000


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2500 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2500 [00:00<?, ?it/s]

In [17]:
evaluate_mmodel_task1(model, df_valid)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1841
           1       0.63      0.63      0.63       300

    accuracy                           0.90      2141
   macro avg       0.78      0.78      0.78      2141
weighted avg       0.90      0.90      0.90      2141



## Model 3 - Trained SBERT with triplet loss

In [18]:
set_reproducibility(base_seed)

# Instantiate model

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [19]:
max_length = 128
train_ds = create_triplet_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                   max_length, n_per_document=20, max_samples = 20000, model=model, margin=0.6, is_task1=True)
print("Dataset size:", len(train_ds))
train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

# Define triplet loss
triplet_loss = losses.TripletLoss(model=model, triplet_margin=0.6, distance_metric=losses.TripletDistanceMetric.COSINE)
epochs = 2

# Train
model.fit(train_objectives=[(train_dl, triplet_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       })

18493
Dataset size: 18493


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2312 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2312 [00:00<?, ?it/s]

In [20]:
evaluate_mmodel_task1(model, df_valid)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1841
           1       0.62      0.62      0.62       300

    accuracy                           0.89      2141
   macro avg       0.78      0.78      0.78      2141
weighted avg       0.89      0.89      0.89      2141



## Model 4 - Trained SBERT with triplet Loss, semi hard mining

In [21]:
set_reproducibility(base_seed)
# Instantiate model

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [22]:
# Define triplet loss
triplet_loss = losses.TripletLoss(model=model, triplet_margin=0.6, distance_metric=losses.TripletDistanceMetric.COSINE)

model.eval()
train_ds = create_triplet_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                            max_length, n_per_document=10, max_samples = 50000, model=model, margin=0.6, 
                                 mine_all=False, mine_semihard=True, mine_hard=True)
train_dl = DataLoader(train_ds,
                  shuffle=True,
                  batch_size=8,
                  collate_fn=model.smart_batching_collate)

epochs = 3
model.train()
# Train
model.fit(train_objectives=[(train_dl, triplet_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       })

N easy:0, N hard: 3402, N semihard:5365
8767


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1096 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1096 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1096 [00:00<?, ?it/s]

In [25]:
evaluate_mmodel_task1(model, df_valid)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1841
           1       0.65      0.65      0.65       300

    accuracy                           0.90      2141
   macro avg       0.80      0.80      0.80      2141
weighted avg       0.90      0.90      0.90      2141



# Task 2

In [10]:
train_directory = './dataset2/train'
df_train = create_dataframe(train_directory)
print(df_train.head())

valid_directory = './dataset2/validation'
df_valid = create_dataframe(valid_directory)
print(df_valid.head())

   authors                site  multi-author  \
0        3  serverfault.com.7z             1   
1        5  serverfault.com.7z             1   
2        5    superuser.com.7z             1   
3        3  serverfault.com.7z             1   
4        1  serverfault.com.7z             0   

                                changes  \
0  [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1]   
1                    [1, 1, 1, 1, 1, 1]   
2              [1, 0, 1, 1, 0, 1, 1, 1]   
3                       [1, 1, 1, 0, 1]   
4                             [0, 0, 0]   

                         paragraph-authors  \
0  [1, 2, 3, 2, 1, 2, 2, 1, 3, 3, 1, 1, 3]   
1                    [1, 2, 3, 4, 5, 2, 3]   
2              [1, 2, 2, 3, 2, 2, 4, 3, 5]   
3                       [1, 2, 3, 1, 1, 2]   
4                             [1, 1, 1, 1]   

                                          input_text  \
0  Small Business Server (SBS) would seem to be a...   
1  None of the suggested key combinations bring m...   
2  If y

In [11]:
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics.pairwise import cosine_distances

def prediction_task2_threshold(model, sentences, threshold):
  embeddings = model.encode(sentences, convert_to_tensor=True)

  pred_authors = [1]
  for i in range(1, len(sentences)):
    similarity_scores = []
    for j in range(i):
        sim = util.cos_sim(embeddings[i], embeddings[j])
        similarity_scores.append(sim.item()) 
    changes = [s > threshold for s in similarity_scores]
    
    if all(x == 0 for x in changes):
        # New author
        new_auth = max(pred_authors)+1
        if new_auth > 5:
            most_similar = pred_authors[np.argmax(similarity_scores)]
            pred_authors.append(most_similar)
        else:
            pred_authors.append(new_auth)
    else:
        # Find most similar
        most_similar = pred_authors[np.argmax(similarity_scores)]
        pred_authors.append(most_similar)
  return pred_authors

def prediction_task2_unsupervised(model, sentences):
  embeddings = model.encode(sentences, convert_to_tensor=False)

  # Compute all pairwise cosine similarities
  sim = cosine_distances(embeddings, embeddings)
  
  documents = np.arange(len(embeddings)).reshape(-1,1)
  clustered = DBSCAN(metric="precomputed", eps=0.5, min_samples=1).fit(sim)
  #clustered = AgglomerativeClustering(affinity="precomputed", n_clusters=row["authors"], linkage="average").fit(sim)
  
  y_pred = [x for x in clustered.labels_]
  y_pred_constr = []
  
  for idx1, el1 in enumerate(y_pred):
    if el1 < 5:
      y_pred_constr.append(el1+1)
    else:
      distance_dict = {}
      for idx2, el2 in enumerate(y_pred):
        if el2 >= 5 or idx1 == idx2:
          continue
        if el2 not in distance_dict:
          distance_dict[el2] = [sim[idx1,idx2]]
        else:
          distance_dict[el2].append(sim[idx1,idx2])
        
      distance_dict = {k:np.mean(distance_dict[k]) for k in distance_dict}
      new_label = min(distance_dict, key=distance_dict.get)
      y_pred_constr.append(new_label+1)
        
  return y_pred_constr
    
def evaluate_mmodel_task2(model, df, threshold):
  valid_labels = []
  y_preds = []

  for i, row in df.iterrows():
    sentences = row["splitted_text"]
    
    if threshold is None:
      pred_authors = prediction_task2_unsupervised(model, sentences)
    else:
      pred_authors = prediction_task2_threshold(model, sentences, threshold)

    valid_labels += row["paragraph-authors"]
    y_preds += pred_authors

  print(classification_report(valid_labels, y_preds))


## Model 1 - Frozen SBERT

In [25]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [26]:
# Create a training DS just to calibrate the threshold
max_length = 128
train_ds = create_contrastive_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                       max_length, max_samples = 50000)
print("Dataset size:", len(train_ds))
train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

model.eval()
# Compute the threshold
same_sims, diff_sims = compute_same_diff_contrastive(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])

print(f"Using {th} as threshold")
evaluate_mmodel_task2(model, df_valid, th)

print(f"Unsupervised clustering")
evaluate_mmodel_task2(model, df_valid, None)

154108
272236
Dataset size: 50000
Using 0.43099294799502624 as threshold
              precision    recall  f1-score   support

           1       0.64      0.77      0.70      5098
           2       0.44      0.49      0.46      2828
           3       0.33      0.26      0.29      1701
           4       0.29      0.11      0.16       978
           5       0.16      0.03      0.04       432

    accuracy                           0.53     11037
   macro avg       0.37      0.33      0.33     11037
weighted avg       0.49      0.53      0.50     11037

Unsupervised clustering
              precision    recall  f1-score   support

           1       0.59      0.83      0.69      5098
           2       0.44      0.39      0.41      2828
           3       0.32      0.19      0.24      1701
           4       0.29      0.10      0.14       978
           5       0.15      0.03      0.05       432

    accuracy                           0.52     11037
   macro avg       0.36      0.31 

## Model 2 - Trained SBERT with contrastive loss

In [12]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/117 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/354 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [13]:
max_length = 128
train_ds = create_contrastive_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                       max_length, max_samples = 50000)
print("Dataset size:", len(train_ds))
train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

# Define contrastive loss
contrastive_loss = losses.ContrastiveLoss(model=model, distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE, margin=0.6)
epochs = 2

# Train
model.fit(train_objectives=[(train_dl, contrastive_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       }
        )

154108
272236
Dataset size: 50000


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6250 [00:00<?, ?it/s]

In [14]:
model.eval()
# Compute the threshold
same_sims, diff_sims = compute_same_diff_contrastive(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])

print(f"Using {th} as threshold")
evaluate_mmodel_task2(model, df_valid, th)

th = 0.6

print(f"Using margin({th}) as threshold")
evaluate_mmodel_task2(model, df_valid, th)

print(f"Unsupervised clustering")
evaluate_mmodel_task2(model, df_valid, None)

Using 0.7385238137675535 as threshold
              precision    recall  f1-score   support

           1       0.74      0.78      0.76      5098
           2       0.56      0.59      0.57      2828
           3       0.40      0.39      0.40      1701
           4       0.33      0.25      0.29       978
           5       0.24      0.17      0.20       432

    accuracy                           0.60     11037
   macro avg       0.45      0.44      0.44     11037
weighted avg       0.59      0.60      0.59     11037

Using margin(0.6) as threshold
              precision    recall  f1-score   support

           1       0.62      0.91      0.74      5098
           2       0.52      0.49      0.51      2828
           3       0.43      0.19      0.26      1701
           4       0.39      0.04      0.08       978
           5       0.43      0.01      0.01       432

    accuracy                           0.58     11037
   macro avg       0.48      0.33      0.32     11037
weighted

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model 3 - Trained SBERT with triplet loss

In [15]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [16]:
max_length = 128
train_ds = create_triplet_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                            max_length, n_per_document=20, max_samples = 50000, model=model, margin=0.6)
train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

# Define triplet loss
triplet_loss = losses.TripletLoss(model=model, triplet_margin=0.6, distance_metric=losses.TripletDistanceMetric.COSINE)
epochs = 2

# Train
model.fit(train_objectives=[(train_dl, triplet_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       })

N easy:49349, N hard: 0, N semihard:0
49349


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6169 [00:00<?, ?it/s]

In [17]:
model.eval()

# Compute the threshold
same_sims, diff_sims = compute_same_diff_triplet(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])
#th = 0.6

print(f"Using {th} as threshold")
evaluate_mmodel_task2(model, df_valid, th)

th = 0.6

print(f"Using margin({th}) as threshold")
evaluate_mmodel_task2(model, df_valid, th)

print(f"Unsupervised clustering")
evaluate_mmodel_task2(model, df_valid, None)

Using 0.37117809966651494 as threshold
              precision    recall  f1-score   support

           1       0.70      0.79      0.74      5098
           2       0.53      0.60      0.56      2828
           3       0.37      0.32      0.35      1701
           4       0.31      0.16      0.21       978
           5       0.19      0.05      0.08       432

    accuracy                           0.58     11037
   macro avg       0.42      0.38      0.39     11037
weighted avg       0.55      0.58      0.56     11037

Using margin(0.6) as threshold
              precision    recall  f1-score   support

           1       0.78      0.66      0.72      5098
           2       0.53      0.55      0.54      2828
           3       0.36      0.41      0.38      1701
           4       0.26      0.30      0.28       978
           5       0.15      0.24      0.19       432

    accuracy                           0.55     11037
   macro avg       0.42      0.43      0.42     11037
weighte

## Model 4 - Trained SBERT with triplet loss and semi hard negative mining

In [28]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [29]:
# Define triplet loss
triplet_loss = losses.TripletLoss(model=model, triplet_margin=0.6, distance_metric=losses.TripletDistanceMetric.COSINE)
model.eval()
train_ds = create_triplet_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                            max_length, n_per_document=10, max_samples = 50000, model=model, margin=0.6, 
                                 mine_all=False, mine_semihard=True, mine_hard=True)
train_dl = DataLoader(train_ds,
                  shuffle=True,
                  batch_size=8,
                  collate_fn=model.smart_batching_collate)

epochs = 3
model.train()
# Train
model.fit(train_objectives=[(train_dl, triplet_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       })

N easy:0, N hard: 9773, N semihard:16076
25849


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3232 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3232 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3232 [00:00<?, ?it/s]

In [35]:
model.eval()
same_sims, diff_sims = compute_same_diff_triplet(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])

print(f"Using {th} as threshold")
evaluate_mmodel_task2(model, df_valid, th)

th = 0.6

print(f"Using margin({th}) as threshold")
evaluate_mmodel_task2(model, df_valid, th)

print(f"Unsupervised clustering")
evaluate_mmodel_task2(model, df_valid, None)

Using 0.3 as threshold
              precision    recall  f1-score   support

           1       0.68      0.79      0.73      5098
           2       0.51      0.59      0.55      2828
           3       0.36      0.29      0.32      1701
           4       0.24      0.09      0.13       978
           5       0.33      0.04      0.07       432

    accuracy                           0.57     11037
   macro avg       0.42      0.36      0.36     11037
weighted avg       0.53      0.57      0.54     11037

Using margin(0.6) as threshold
              precision    recall  f1-score   support

           1       0.79      0.61      0.69      5098
           2       0.53      0.54      0.53      2828
           3       0.35      0.41      0.38      1701
           4       0.22      0.31      0.25       978
           5       0.13      0.25      0.17       432

    accuracy                           0.52     11037
   macro avg       0.40      0.42      0.41     11037
weighted avg       0.58

# Task 3

In [12]:
train_directory = './dataset3/train'
df_train = create_dataframe(train_directory)
print(df_train.head())

valid_directory = './dataset3/validation'
df_valid = create_dataframe(valid_directory)
print(df_valid.head())

we have a problem at row:  1194
   authors                                   site  multi-author  \
0        5                     serverfault.com.7z             1   
1        1           gamedev.stackexchange.com.7z             0   
2        5                       superuser.com.7z             1   
3        1  computergraphics.stackexchange.com.7z             0   
4        3                       superuser.com.7z             1   

                                             changes  \
0                  [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1]   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2         [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1]   
3                                 [0, 0, 0, 0, 0, 0]   
4  [1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, ...   

                                   paragraph-authors  \
0               [1, 2, 1, 1, 3, 2, 2, 3, 4, 5, 5, 2]   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2      [1, 2, 3, 1, 1, 4, 5, 5, 1, 3, 3, 2, 2, 2, 1]   
3   

In [13]:
def prediction_task3_threshold(model, sentences, threshold):
  embeddings = model.encode(sentences, convert_to_tensor=True)

  similarity_scores = []

  for i in range(len(sentences)-1):
    sim = util.cos_sim(embeddings[i], embeddings[i+1])
    similarity_scores.append(sim.item()) 
  
  pred_changes = [1 if sim < th else 0 for sim in similarity_scores]
  return pred_changes
    
def evaluate_mmodel_task3(model, df, threshold):
  valid_labels = []
  y_preds = []

  for i, row in df.iterrows():
    sentences = row["splitted_text"]
    
    pred_authors = prediction_task3_threshold(model, sentences, threshold)

    valid_labels += row["changes"]
    y_preds += pred_authors

  print(classification_report(valid_labels, y_preds))

## Model 1 - Frozen SBERT

In [13]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [14]:
# Create a training DS just to calibrate the threshold
max_length = 32
train_ds = create_contrastive_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer,
                                       max_length, max_samples = 500000)
print("Dataset size:", len(train_ds))
train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

model.eval()
# Compute the threshold
same_sims, diff_sims = compute_same_diff_contrastive(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])

print(f"Using {th} as threshold")
evaluate_mmodel_task3(model, df_valid, th)


928658
1186486
Dataset size: 500000
Using 0.3619453541844835 as threshold
              precision    recall  f1-score   support

           0       0.53      0.65      0.59     10043
           1       0.64      0.53      0.58     12062

    accuracy                           0.58     22105
   macro avg       0.59      0.59      0.58     22105
weighted avg       0.59      0.58      0.58     22105



## Model 2 - Trained SBERT with contrastive loss

In [14]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/117 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/354 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [15]:
max_length = 32
train_ds = create_contrastive_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                       max_length, max_samples = 100000)
print("Dataset size:", len(train_ds))
train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

# Define contrastive loss
contrastive_loss = losses.ContrastiveLoss(model=model, distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE, margin=0.6)
epochs = 2

# Train
model.fit(train_objectives=[(train_dl, contrastive_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       }
        )

928658
1186486
Dataset size: 100000


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12500 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12500 [00:00<?, ?it/s]

In [16]:
model.eval()
# Compute the threshold
same_sims, diff_sims = compute_same_diff_contrastive(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])

print(f"Using {th} as threshold")
evaluate_mmodel_task3(model, df_valid, th)

th = 0.6

print(f"Using margin({th}) as threshold")
evaluate_mmodel_task3(model, df_valid, th)


Using 0.6878071384977197 as threshold
              precision    recall  f1-score   support

           0       0.65      0.63      0.64     10043
           1       0.70      0.71      0.71     12062

    accuracy                           0.68     22105
   macro avg       0.67      0.67      0.67     22105
weighted avg       0.68      0.68      0.68     22105

Using margin(0.6) as threshold
              precision    recall  f1-score   support

           0       0.58      0.81      0.68     10043
           1       0.76      0.51      0.61     12062

    accuracy                           0.65     22105
   macro avg       0.67      0.66      0.64     22105
weighted avg       0.68      0.65      0.64     22105



## Model 3 - Trained SBERT with triplet loss

In [20]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [21]:
max_length = 32
train_ds = create_triplet_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                            max_length, n_per_document=20, max_samples = 100000, model=model, margin=0.6)
train_dl = DataLoader(train_ds,
                      shuffle=True,
                      batch_size=8,
                      collate_fn=model.smart_batching_collate)

# Define triplet loss
triplet_loss = losses.TripletLoss(model=model, triplet_margin=0.6, distance_metric=losses.TripletDistanceMetric.COSINE)
epochs = 2

# Train
model.fit(train_objectives=[(train_dl, triplet_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       })

N easy:96005, N hard: 0, N semihard:0
96005


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12001 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12001 [00:00<?, ?it/s]

In [22]:
model.eval()

# Compute the threshold
same_sims, diff_sims = compute_same_diff_triplet(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])

print(f"Using {th} as threshold")
evaluate_mmodel_task3(model, df_valid, th)

th = 0.6

print(f"Using margin({th}) as threshold")
evaluate_mmodel_task3(model, df_valid, th)


Using 0.2701155882739125 as threshold
              precision    recall  f1-score   support

           0       0.62      0.63      0.63     10043
           1       0.69      0.68      0.68     12062

    accuracy                           0.66     22105
   macro avg       0.66      0.66      0.66     22105
weighted avg       0.66      0.66      0.66     22105

Using margin(0.6) as threshold
              precision    recall  f1-score   support

           0       0.71      0.37      0.48     10043
           1       0.62      0.88      0.73     12062

    accuracy                           0.64     22105
   macro avg       0.67      0.62      0.61     22105
weighted avg       0.66      0.64      0.62     22105



## Model 4 - Trained SBERT with triplet loss and semi hard negative mining

In [16]:
set_reproducibility(base_seed)

#model_type = "all-mpnet-base-v2"
model_type = 'AnnaWegmann/Style-Embedding'

model = SentenceTransformer(model_type)

In [17]:
# Define triplet loss
triplet_loss = losses.TripletLoss(model=model, triplet_margin=0.6, distance_metric=losses.TripletDistanceMetric.COSINE)

max_length = 32
train_ds = create_triplet_examples(df_train["splitted_text"].values, df_train["paragraph-authors"].values, model.tokenizer, 
                                            max_length, n_per_document=10, max_samples = 100000, model=model, margin=0.6, 
                                 mine_all=False, mine_semihard=True, mine_hard=True)
train_dl = DataLoader(train_ds,
                  shuffle=True,
                  batch_size=16,
                  collate_fn=model.smart_batching_collate)

epochs = 3

# Train
model.fit(train_objectives=[(train_dl, triplet_loss)],
                       epochs=epochs,
                       warmup_steps=math.ceil(len(train_dl) * epochs * 0.1),
                       optimizer_params={
                           'lr': 2e-5,
                       })

N easy:0, N hard: 20598, N semihard:24095
44693


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5587 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5587 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5587 [00:00<?, ?it/s]

In [18]:
model.eval()
same_sims, diff_sims = compute_same_diff_triplet(model, train_dl)
th = np.mean([np.mean(same_sims)-2*np.std(same_sims), np.mean(diff_sims)+2*np.std(diff_sims)])
#th = 0.6

print(f"Using {th} as threshold")
evaluate_mmodel_task3(model, df_valid, th)

th = 0.6

print(f"Using margin({th}) as threshold")
evaluate_mmodel_task3(model, df_valid, th)

Using 0.23284846905517453 as threshold
              precision    recall  f1-score   support

           0       0.60      0.62      0.61     10043
           1       0.68      0.66      0.67     12062

    accuracy                           0.64     22105
   macro avg       0.64      0.64      0.64     22105
weighted avg       0.64      0.64      0.64     22105

Using margin(0.6) as threshold
              precision    recall  f1-score   support

           0       0.71      0.33      0.45     10043
           1       0.61      0.89      0.73     12062

    accuracy                           0.63     22105
   macro avg       0.66      0.61      0.59     22105
weighted avg       0.66      0.63      0.60     22105

